<a href="https://colab.research.google.com/github/ChenYu-K/Data-Processing/blob/main/Neural%20Network/bolt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset, DataLoader
import os
import cv2 as cv

In [24]:
root_path='/content/drive/MyDrive/datebase/bolt'

In [69]:
names=os.listdir('/content/drive/MyDrive/datebase/bolt/train')
filenames=[]
for name in names:
  if os.path.splitext(name)[1] == '.png':
    fname=os.path.splitext(name)[0]
    filenames.append(fname)
print(filenames)

['125', '124', '122', '123', '119', '120', '12', '121', '118', '117', '116', '115', '111', '113', '11', '114', '109', '110', '112', '108', '107', '106', '105', '103', '10', '102', '101', '104', '100', '1', '0', '164', '161', '160', '158', '162', '157', '159', '156', '163', '155', '16', '154', '152', '149', '150', '146', '151', '145', '147', '15', '144', '140', '137', '138', '139', '134', '14', '143', '141', '136', '135', '127', '128', '13', '132', '130', '129', '131', '133', '126', '22', '193', '198', '195', '194', '199', '192', '197', '196', '20', '2', '185', '190', '191', '186', '19', '187', '188', '189', '184', '183', '182', '181', '179', '176', '175', '18', '178', '180', '174', '177', '172', '170', '169', '168', '17', '165', '167', '173', '166', '56', '48', '52', '53', '50', '51', '49', '54', '5', '55', '47', '37', '42', '45', '43', '4', '40', '41', '46', '39', '38', '29', '3', '31', '32', '28', '33', '34', '30', '26', '25', '201', '24', '202', '204', '205', '203', '200', '21', '23

# 读取图片并打标签
用cv读取，经过基本的预处理（比如resize）后，转成pimg

In [76]:
def generate_dataset(dir, set_label):
    """
    set_label should be 'torch.tensor([1])' if two-catogory and positive sample
    """
    train_data = []
    labels = []
    for file_name in os.listdir(dir):
        img_dir = os.path.join(dir, file_name)
        img = cv.imread(img_dir)
        #img = cv.resize(img, (120, 120))            # resize img
        #pimg = Image.fromarray(img)
        train_data.append(img)
        labels.append(set_label)
    return train_data, labels

In [77]:
train_data, train_label = generate_dataset('/content/drive/MyDrive/datebase/bolt/train',filenames)
valid_data, valid_label = generate_dataset('/content/drive/MyDrive/datebase/bolt/test',filenames)

# 重写dataset类

In [79]:
class MyDataset(Dataset):
    def __init__(self, data, labels, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        imgs = []
        for i in range(len(labels)):
            # print(type(data[i]))    # <class 'PIL.Image.Image'>
            im_tensor = transform(data[i]).to(torch.device("cpu"))
            imgs.append((im_tensor, labels[i]))
        self.imgs = imgs                         # DataLoader通过getitem读取图片数据
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        return fn, label
    def __len__(self):
        return len(self.imgs)
#原文链接：https://blog.csdn.net/csdntcl/article/details/121107971

# 用MyDataset构建数据集

In [ ]:
normMean = [0.35174567, 0.4027052, 0.39986762]
normStd = [0.18738173, 0.17127964, 0.1971462]
transform = transforms.Compose(
    [transforms.ToTensor(),  # range [0, 255] -> [0.0,1.0]   
    transforms.Normalize(normMean, normStd)
    ])
# 也可以再定义train_transform加入一些数据增强 
train_data = MyDataset(train_data, train_labels, transform=transform)
valid_data = MyDataset(valid_data, valid_labels, transform=transform)
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE, shuffle=True)
————————————————
版权声明：本文为CSDN博主「火星程猿」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/csdntcl/article/details/121107971